# Python - Multiprocessing

- Introducing Python: Modern Computing in Simple Packages (p.269)
- [Queue](https://www.youtube.com/watch?v=_TNIlBlV5c0)

In [1]:
import multiprocessing
import os
import time

In [2]:
def do_job(name):
    whoami(name)
    
def whoami(name):
    print(f'Process {os.getpid()} doing {name}')
    
def loopy(name):
    whoami(name)
    start = 1
    stop = 100000
    for i in range(start, stop):
        print(f'Doing {i} of {stop}')
        time.sleep(1)

In [13]:
if __name__ == '__main__':
    whoami('Main program')
    for i in range(4):
        p = multiprocessing.Process(target=do_job, args=(f'loop {i}',))
        p.start()

Process 2919 doing Main program
Process 2945 doing loop 0
Process 2946 doing loop 1
Process 2947 doing loop 2
Process 2948 doing loop 3


In [23]:
if __name__ == '__main__':
    whoami('Main program')
    p = multiprocessing.Process(target=loopy, args=(f'loopy',))
    p.start()  # 開始背景作業
    print('Backgound processing...')
    time.sleep(5)
    p.terminate()  # 提前終止
    print('Stopped')

Process 2919 doing Main program
Process 2970 doing loopy
Doing 1 of 100000
Backgound processing...
Doing 2 of 100000
Doing 3 of 100000
Doing 4 of 100000
Doing 5 of 100000
Stopped


---

## Queue

In [7]:
import multiprocessing as mp
import time
import os

In [8]:
def washer(dishes, output):
    for dish in dishes:
        print(f'[{time.time() - tic:.2f}] Washing dish ({os.getpid()}): {dish}')
        time.sleep(1)
        output.put(dish)
        
def dryer(input):
    while True:
        dish = input.get()
        print(f'[{time.time() - tic:.2f}] Drying dish ({os.getpid()}): {dish}')
        time.sleep(5)
        input.task_done()

In [11]:
if __name__ == '__main__':
    tic = time.time()
    dish_queue = mp.JoinableQueue()
    
    # 逐個加載 Processor 到 dish_queue
    dryer_proc1 = mp.Process(target=dryer, args=(dish_queue,))
    dryer_proc2 = mp.Process(target=dryer, args=(dish_queue,))
    dryer_proc1.daemon = True  # 會在背景一直等著執行 This must be set before start() is called.
    dryer_proc2.daemon = True  # 總共 2 dryers
    dryer_proc1.start()  # 開始進程 1
    dryer_proc2.start()  # 開始進程 2

    # 開始工作
    dishes = ['apple', 'banana', 'orange', 'salad']
    washer(dishes, dish_queue)  # washer 先逐個洗盤子，洗完的盤子丟到 dish_queue
    dish_queue.join()  # Block until all items in the queue have been gotten and processed.

[0.04] Washing dish (8840): apple
[1.05] Drying dish (8914): apple
[1.05] Washing dish (8840): banana
[2.06] Drying dish (8915): banana
[2.06] Washing dish (8840): orange
[3.06] Washing dish (8840): salad
[6.06] Drying dish (8914): orange
[7.07] Drying dish (8915): salad


[**Queue vs JoinableQueue**](https://stackoverflow.com/a/31230329/3744499)

JoinableQueue has methods join() and task_done(), which Queue hasn't.

> **class multiprocessing.JoinableQueue( [maxsize] )**
> 
> JoinableQueue, a Queue subclass, is a queue which additionally has task_done() and join() methods.

If you use `JoinableQueue` then you must call `JoinableQueue.task_done()` for each task removed from the queue or else the semaphore used to count the number of unfinished tasks may eventually overflow, raising an exception.